In [1]:
import pandas as pd 
import numpy as np

In [2]:
raw_sessiondata = pd.read_csv("20210113-133853.csv", skiprows=6, delimiter = ';')

In [3]:
#Add a column for 'timestamps'
import time
import datetime
import ciso8601

timestamp = []
for i in range(len(raw_sessiondata.index)):
    t = raw_sessiondata['PC-TIME'].iloc[i]
    ts= ciso8601.parse_datetime(t)
    if i == 0:
        t0= time.mktime(ts.timetuple())+ts.microsecond/1000000
        timestamp.append(np.around(t0-t0,3))
    else:
        tnow = time.mktime(ts.timetuple())+ts.microsecond/1000000
        timestamp.append(np.around(tnow-t0,3))

raw_sessiondata['timestamps'] = timestamp

In [4]:
df_group_info = raw_sessiondata.groupby('+INFO')
df_group_type = raw_sessiondata.groupby('TYPE')

## Lick Response Timestamps

In [9]:
# Get lick onsets 
df_port1out = df_group_info.get_group('Port1Out')
timestamps_lickon = df_port1out.timestamps.to_numpy()

In [10]:
timestamps_10ms= timestamps_lickon*100

In [8]:
rounded_timestamps_10ms = [int(round(timestamps_10ms[i])) for i in range(len(timestamps_10ms))]
print(rounded_timestamps_10ms[0:10])

[791, 819, 854, 872, 1675, 1696, 2161, 2175, 2194, 2208]


In [17]:
sampled_lick_10ms = np.zeros(rounded_timestamps_10ms[-1]+1)
np.put(sampled_lick_10ms,rounded_timestamps_10ms,1)

#### Checking Accuracy; should have the result as 'rounded_timestamps_10ms'

In [19]:
np.where(sampled_lick_10ms == 1)

(array([   791,    819,    854, ..., 230803, 230820, 230830]),)

## Creating DataFrame: 'Sampled_10ms.lick'

In [20]:
# intialise data of lists. 
data = {'Sampled_10ms.lick':sampled_lick_10ms } 
# Create DataFrame 
df_licks = pd.DataFrame(data)

In [21]:
df_licks

,Sampled_10ms.lick
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
230826,0.0
230827,0.0
230828,0.0
230829,0.0


**Saving to h5**

In [22]:
df_licks.to_hdf('Sampled_10ms.licks.h5', key = 'dflicks')